In [ ]:
import pandas as pd

file_path = 'originalDataset.csv'


df = pd.read_csv(file_path, delimiter='\t')

# número inicial de linhas
initial_row_count = len(df)
print(f"Número inicial de linhas no originalDataset: {initial_row_count}")


df.replace('NONE', pd.NA, inplace=True)

# eliminar as linhas com NaN (valores vazios) em qualquer coluna
df.dropna(inplace=True)

#  critérios de filtragem
criteria = {
    'education': (1, 4),
    'gender': (1, 3),
    'married': (1, 3),
    'age': (18, 100)
}

for column, (low, high) in criteria.items():
    df = df[df[column].between(low, high)]

# participantes por país
country_counts = df['country'].value_counts()

# países com 30 ou mais participantes
countries_with_min_participants = country_counts[country_counts >= 30].index

# apenas os países com 30 ou mais participantes
df_filtered = df[df['country'].isin(countries_with_min_participants)]

#  linhas após os filtros
filtered_row_count = len(df_filtered)
print(f"Número de linhas após todos os filtros: {filtered_row_count}")

# -1 das questões
item_columns = [
    'Q3A', 'Q5A', 'Q10A', 'Q13A', 'Q16A', 'Q17A',
    'Q21A', 'Q24A', 'Q26A', 'Q31A', 'Q34A', 'Q37A',
    'Q38A', 'Q42A'
]
df_filtered[item_columns] = df_filtered[item_columns] - 1

# Adicionar a coluna soma_depressao
df_filtered['soma_depressao'] = df_filtered[item_columns].sum(axis=1)

# indice de depressão
def mapear_indice_depressao(pontuacao):
    if pontuacao <= 9:
        return 0
    elif pontuacao <= 13:
        return 1
    elif pontuacao <= 20:
        return 2
    elif pontuacao <= 27:
        return 3
    else:
        return 4

df_filtered['indice_depressao'] = df_filtered['soma_depressao'].apply(mapear_indice_depressao)


# paises escolhidos
country_legend = {number: country for number, country in enumerate(df_filtered['country'].unique(), 1)}
print("Legenda para a coluna 'country':")
for number, country in country_legend.items():
    print(f"{number}: {country}")

# Remover a coluna 'soma_depressao' antes de salvar o dataset final
df_filtered.drop(columns=['soma_depressao'], inplace=True)


df_filtered.to_csv('dataset.csv', index=False)

final_row_count = len(df_filtered)
print(f"Número de linhas finais no dataset filtrado: {final_row_count}")


Número inicial de linhas no originalDataset: 39775
Número de linhas após todos os filtros: 24927
Legenda para a coluna 'country':
1: MY
2: GB
3: DE
4: US
5: AU
6: CA
7: FI
8: ES
9: ID
10: GR
11: IN
12: BR
13: PT
14: RS
15: IT
16: FR
17: RO
18: PL
19: PH
20: IE
21: NZ
22: ZA
23: NL
24: JM
25: MX
26: PK
27: HK
28: SG
29: RU
30: TR
31: EG
32: SA
33: BN
34: JP


<ipython-input-34-5198658f45d1>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[item_columns] = df_filtered[item_columns] - 1
<ipython-input-34-5198658f45d1>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['soma_depressao'] = df_filtered[item_columns].sum(axis=1)
<ipython-input-34-5198658f45d1>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Número de linhas finais no dataset filtrado: 24927


In [2]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd


df = pd.read_csv('dataset.csv')
# encontra  colunas categorizáveis (objetos ou inteiros que representam categorias)
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()


print(f"Colunas categorizáveis identificadas para One-Hot Encoding: {categorical_columns}")

# Remover 'indice_depressao' do conjunto de colunas categorizáveis
categorical_columns = [col for col in categorical_columns if col != 'indice_depressao']

# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# adicionar 'indice_depressao' de volta
df_encoded['indice_depressao'] = df['indice_depressao']

# salvar o dataset codificado
df_encoded.to_csv('dataset_one_hot_encoded.csv', index=False)

# mostrar o número de linhas e colunas no dataset final
print(f"Número de linhas no dataset codificado: {len(df_encoded)}")
print(f"Número de colunas no dataset codificado: {len(df_encoded.columns)}")


Colunas categorizáveis identificadas para One-Hot Encoding: ['major']
Número de linhas no dataset codificado: 24927
Número de colunas no dataset codificado: 4894


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
import json

# dados normalizados
data = pd.read_csv('dataset_one_hot_encoded.csv')

# variáveis independentes e dependente
X = data.drop(columns=['indice_depressao'])
y = data['indice_depressao']

# (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalização Min-Max
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

# modelos conf padrão
modelos = {
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Regressão Logística': LogisticRegression(),
    'K-Vizinhos Mais Próximos': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Árvore de Decisão': DecisionTreeClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'CatBoost': CatBoostClassifier(),
    'XGBoost': XGBClassifier(),
}

# Validação cruzada e treinamento
resultados = {}
matrizes_confusao = {}
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for nome, modelo in modelos.items():
    scores = []
    matrizes = np.zeros((len(np.unique(y)), len(np.unique(y))))
    for train_idx, test_idx in skf.split(X_train_scaled, y_train):
        X_train_fold, X_test_fold = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]

        # Treinamento
        modelo.fit(X_train_fold, y_train_fold)
        y_pred = modelo.predict(X_test_fold)

        accuracy = accuracy_score(y_test_fold, y_pred)
        scores.append(accuracy)

        matrizes += confusion_matrix(y_test_fold, y_pred)

    # Média das acurácias e matriz normalizada
    resultados[nome] = round(np.mean(scores), 4)
    matrizes_confusao[nome] = matrizes / skf.get_n_splits()

    print(f"{nome} - Acurácia média (Validação Cruzada 10 Folds): {resultados[nome]:.4f}")

# Salvar os resultados no JSON
with open('metricas_modelos.json', 'w') as f:
    json.dump(resultados, f)


Random Forest - Acurácia média (Validação Cruzada 10 Folds): 0.8284
SVM - Acurácia média (Validação Cruzada 10 Folds): 0.8775


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

A saída de streaming foi truncada nas últimas 5000 linhas.
6:	learn: 0.9879864	total: 532ms	remaining: 1m 15s
7:	learn: 0.9427403	total: 606ms	remaining: 1m 15s
8:	learn: 0.9044259	total: 677ms	remaining: 1m 14s
9:	learn: 0.8698595	total: 748ms	remaining: 1m 14s
10:	learn: 0.8294274	total: 817ms	remaining: 1m 13s
11:	learn: 0.7990953	total: 884ms	remaining: 1m 12s
12:	learn: 0.7664624	total: 962ms	remaining: 1m 13s
13:	learn: 0.7370696	total: 1.03s	remaining: 1m 12s
14:	learn: 0.7115464	total: 1.1s	remaining: 1m 12s
15:	learn: 0.6904334	total: 1.18s	remaining: 1m 12s
16:	learn: 0.6628448	total: 1.26s	remaining: 1m 12s
17:	learn: 0.6427203	total: 1.33s	remaining: 1m 12s
18:	learn: 0.6242216	total: 1.4s	remaining: 1m 12s
19:	learn: 0.6045621	total: 1.47s	remaining: 1m 12s
20:	learn: 0.5845890	total: 1.54s	remaining: 1m 11s
21:	learn: 0.5668949	total: 1.61s	remaining: 1m 11s
22:	learn: 0.5515127	total: 1.69s	remaining: 1m 11s
23:	learn: 0.5393160	total: 1.75s	remaining: 1m 11s
24:	learn: 

In [6]:
import json

# Carregar o arquivo JSON
with open('metricas_modelos.json', 'r') as f:
    metricas_modelos = json.load(f)

# Exibir as métricas no console
print("Resultados das Acurácias Médias:")
for modelo, acuracia in metricas_modelos.items():
    print(f"{modelo}: {acuracia:.4f}")


Resultados das Acurácias Médias:
Random Forest: 0.8284
SVM: 0.8775
Regressão Logística: 0.8603
K-Vizinhos Mais Próximos: 0.7598
Naive Bayes: 0.7794
Árvore de Decisão: 0.7868
AdaBoost: 0.7879
CatBoost: 0.8579
XGBoost: 0.8359


In [ ]:
import pandas as pd

data = pd.read_csv('dataset.csv')

# mapear iso do dataset
country_name_dict = {
    'AU': 'Australia', 'BN': 'Brunei', 'BR': 'Brazil', 'CA': 'Canada', 'DE': 'Germany', 'EG': 'Egypt',
    'ES': 'Spain', 'FI': 'Finland', 'FR': 'France', 'GB': 'United Kingdom', 'GR': 'Greece', 'HK': 'Hong Kong',
    'ID': 'Indonesia', 'IE': 'Ireland', 'IN': 'India', 'IT': 'Italy', 'JM': 'Jamaica', 'JP': 'Japan',
    'MX': 'Mexico', 'MY': 'Malaysia', 'NL': 'Netherlands', 'NZ': 'New Zealand', 'PH': 'Philippines',
    'PK': 'Pakistan', 'PL': 'Poland', 'PT': 'Portugal', 'RO': 'Romania', 'RS': 'Serbia', 'RU': 'Russia',
    'SA': 'Saudi Arabia', 'SG': 'Singapore', 'TR': 'Turkey', 'US': 'United States', 'ZA': 'South Africa'
}

# Dicionário para mapear os países para seus respectivos continentes
continent_dict = {
    'AU': 'Australia/New Zealand', 'BN': 'Asia', 'BR': 'South America', 'CA': 'North America', 'DE': 'Europe',
'EG': 'Africa', 'ES': 'Europe', 'FI': 'Europe', 'FR': 'Europe', 'GB': 'Europe', 'GR': 'Europe',
    'HK': 'Asia', 'ID': 'Asia', 'IE': 'Europe', 'IN': 'Asia', 'IT': 'Europe', 'JM': 'North America',
        'JP': 'Asia', 'MX': 'North America', 'MY': 'Asia', 'NL': 'Europe', 'NZ': 'Australia/New Zealand',
    'PH': 'Asia', 'PK': 'Asia', 'PL': 'Europe', 'PT': 'Europe', 'RO': 'Europe', 'RS': 'Europe',
    'RU': 'Europe', 'SA': 'Asia', 'SG': 'Asia', 'TR': 'Asia', 'US': 'North America', 'ZA': 'Africa'
}

# testa se as colunas country e indice_depressao estão no dataset
if 'country' in data.columns and 'indice_depressao' in data.columns:
    # Adicionar a coluna 'continent' diretamente a partir dos códigos ISO
    data['continent'] = data['country'].map(continent_dict)

    # verificar se o mapeamento foi realizado corretamente
    print("Exemplo de mapeamento de países para continentes:")
    print(data[['country', 'continent']].head())

    # contagem /continente
    continent_counts = data['continent'].value_counts()

    for continent in continent_counts.index:
        print(f"\nContinente: {continent}, Total de participantes: {continent_counts[continent]}")
        countries_in_continent = data[data['continent'] == continent]['country'].value_counts()

        continent_avg_depression = 0
        country_count = 0

        for country_code, count in countries_in_continent.items():
            avg_depression = data[data['country'] == country_code]['indice_depressao'].mean()
            country_name = country_name_dict[country_code]  # Obter o nome do país
            print(f"  País: {country_name}, Participantes: {count}, Média do Índice de Depressão: {avg_depression:.2f}")

            # Somar para calcular a média do continente
            continent_avg_depression += avg_depression
            country_count += 1

        continent_avg_depression /= country_count
        print(f"Média do Índice de Depressão no Continente ({continent}): {continent_avg_depression:.2f}")

    # Exibir a soma total de todos os participantes
    total_participants = continent_counts.sum()
    print(f"\nTotal de participantes em todos os continentes: {total_participants}")

else:
    print("As colunas 'country' ou 'indice_depressao' não foram encontradas no dataset.")


Exemplo de mapeamento de países para continentes:
  country      continent
0      MY           Asia
1      GB         Europe
2      DE         Europe
3      US  North America
4      US  North America

Continente: Asia, Total de participantes: 19180
  País: Malaysia, Participantes: 17252, Média do Índice de Depressão: 2.12
  País: Indonesia, Participantes: 642, Média do Índice de Depressão: 2.32
  País: Philippines, Participantes: 472, Média do Índice de Depressão: 2.46
  País: India, Participantes: 331, Média do Índice de Depressão: 2.23
  País: Singapore, Participantes: 123, Média do Índice de Depressão: 2.08
  País: Brunei, Participantes: 97, Média do Índice de Depressão: 2.18
  País: Japan, Participantes: 74, Média do Índice de Depressão: 1.84
  País: Turkey, Participantes: 54, Média do Índice de Depressão: 2.39
  País: Pakistan, Participantes: 53, Média do Índice de Depressão: 2.32
  País: Saudi Arabia, Participantes: 43, Média do Índice de Depressão: 1.91
  País: Hong Kong, Partic

In [ ]:
Exemplo de mapeamento de países para continentes:
  country      continent
0      MY           Asia
1      GB         Europe
2      DE         Europe
3      US  North America
4      US  North America

Continente: Asia, Total de participantes: 19180
  País: Malaysia, Participantes: 17252, Média do Índice de Depressão: 2.12
  País: Indonesia, Participantes: 642, Média do Índice de Depressão: 2.32
  País: Philippines, Participantes: 472, Média do Índice de Depressão: 2.46
  País: India, Participantes: 331, Média do Índice de Depressão: 2.23
  País: Singapore, Participantes: 123, Média do Índice de Depressão: 2.08
  País: Brunei, Participantes: 97, Média do Índice de Depressão: 2.18
  País: Japan, Participantes: 74, Média do Índice de Depressão: 1.84
  País: Turkey, Participantes: 54, Média do Índice de Depressão: 2.39
  País: Pakistan, Participantes: 53, Média do Índice de Depressão: 2.32
  País: Saudi Arabia, Participantes: 43, Média do Índice de Depressão: 1.91
  País: Hong Kong, Participantes: 39, Média do Índice de Depressão: 2.13
Média do Índice de Depressão no Continente (Asia): 2.18

Continente: North America, Total de participantes: 3813
  País: United States, Participantes: 3277, Média do Índice de Depressão: 2.09
  País: Canada, Participantes: 428, Média do Índice de Depressão: 2.12
  País: Mexico, Participantes: 75, Média do Índice de Depressão: 2.25
  País: Jamaica, Participantes: 33, Média do Índice de Depressão: 2.24
Média do Índice de Depressão no Continente (North America): 2.18

Continente: Europe, Total de participantes: 1322
  País: United Kingdom, Participantes: 517, Média do Índice de Depressão: 2.16
  País: Germany, Participantes: 134, Média do Índice de Depressão: 2.26
  País: France, Participantes: 125, Média do Índice de Depressão: 2.28
  País: Poland, Participantes: 85, Média do Índice de Depressão: 2.69
  País: Italy, Participantes: 72, Média do Índice de Depressão: 2.50
  País: Romania, Participantes: 71, Média do Índice de Depressão: 2.01
  País: Netherlands, Participantes: 59, Média do Índice de Depressão: 2.24
  País: Spain, Participantes: 43, Média do Índice de Depressão: 1.81
  País: Ireland, Participantes: 39, Média do Índice de Depressão: 2.26
  País: Russia, Participantes: 39, Média do Índice de Depressão: 2.31
  País: Greece, Participantes: 37, Média do Índice de Depressão: 2.03
  País: Serbia, Participantes: 35, Média do Índice de Depressão: 1.46
  País: Portugal, Participantes: 34, Média do Índice de Depressão: 2.12
  País: Finland, Participantes: 32, Média do Índice de Depressão: 2.34
Média do Índice de Depressão no Continente (Europe): 2.18

Continente: Australia/New Zealand, Total de participantes: 400
  País: Australia, Participantes: 278, Média do Índice de Depressão: 2.11
  País: New Zealand, Participantes: 122, Média do Índice de Depressão: 2.21
Média do Índice de Depressão no Continente (Australia/New Zealand): 2.16

Continente: Africa, Total de participantes: 107
  País: Egypt, Participantes: 55, Média do Índice de Depressão: 2.58
  País: South Africa, Participantes: 52, Média do Índice de Depressão: 2.13
Média do Índice de Depressão no Continente (Africa): 2.36

Continente: South America, Total de participantes: 105
  País: Brazil, Participantes: 105, Média do Índice de Depressão: 2.20
Média do Índice de Depressão no Continente (South America): 2.20

Total de participantes em todos os continentes: 24927

In [ ]:
# nome no GeoJSON
country_names_from_geojson = [feature['properties']['name'] for feature in geojson_data['features']]
print(country_names_from_geojson)



['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates', 'Argentina', 'Armenia', 'Antarctica', 'French Southern and Antarctic Lands', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'The Bahamas', 'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Brunei', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Switzerland', 'Chile', 'China', 'Ivory Coast', 'Cameroon', 'Democratic Republic of the Congo', 'Republic of the Congo', 'Colombia', 'Costa Rica', 'Cuba', 'Northern Cyprus', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji', 'Falkland Islands', 'France', 'Gabon', 'United Kingdom', 'Georgia', 'Ghana', 'Guinea', 'Gambia', 'Guinea Bissau', 'Equatorial Guinea', 'Greece', 'Greenland', 'Guatemala', 'French Guiana', 'Guyana', 'Honduras', 'Croatia', 'Haiti', 'Hunga

In [ ]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.23.200.150


In [ ]:
!npm install -g localtunnel@2.0.2

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 22 packages in 2s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.200.150:8501

⠼⠴⠦your url is: https://shiny-otters-bake.loca.lt
2024-12-31 23:43:26.165 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-12-31 23:43:35.112 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
2024-12-31 23:43:59.101 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
  Stopping...
^C
